In [1]:
access_key = ''
access_secret = ''
consumer_key = 'qoeSeYEDM3U01KjilFiPd0DcG'
consumer_secret = ''

In [ ]:
import tweepy #https://github.com/tweepy/tweepy
import json

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [ ]:
user = api.get_user('NicolaSturgeon')

In [ ]:
print(user.screen_name)

In [ ]:
print(user.followers_count)

In [ ]:
print(user.description)

In [ ]:
user

In [ ]:
new_tweets = api.user_timeline(screen_name = 'NicolaSturgeon',count=2)

In [ ]:
new_tweets[0]

In [ ]:
brexit_tweets = api.search(q='brexit', lang='en', count='10')

In [ ]:
brexit_tweets[0]

In [ ]:
brexit_tweets[0]._json

In [ ]:
brexit_tweets[0]._json['text'] 

In [ ]:
tweets_text = []
for each in brexit_tweets:
    tweets_text.append(each._json['text'])

In [ ]:
tweets_text

In [ ]:
tweets_string = " ".join(tweets_text)
from nltk.tokenize import word_tokenize
tokens = word_tokenize(tweets_string)

In [ ]:
print(tokens[0:10])

In [ ]:
lowercase_tokens = [token.lower() for token in tokens]

In [ ]:
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
remove_these = set(stopwords.words('english') + list(string.punctuation) + list(string.digits))
filtered_text = [word 
                 for word in lowercase_tokens 
                 if not word in remove_these]
from collections import Counter
simple_frequencies_dict = Counter(filtered_text)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

cloud = WordCloud(max_font_size=80,colormap="hsv").generate_from_frequencies(simple_frequencies_dict)
plt.figure(figsize=(16,12))
plt.imshow(cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
covid_tweets = api.search(q='covid OR covid19 OR COVID OR COVID19 or #covid', lang='en', count='100')

In [ ]:
covid_tweets[0]._json['text'] 

In [ ]:
covid_hashtags = []
for each in covid_tweets:
   for hashtag in each._json['entities']['hashtags']:
    covid_hashtags.append(hashtag['text'])

In [ ]:
hashtag_string = " ".join(covid_hashtags)
tokens = word_tokenize(hashtag_string)
simple_frequencies_dict_covid = Counter(tokens)
cloud = WordCloud(max_font_size=80, colormap="viridis", background_color='white',).generate_from_frequencies(simple_frequencies_dict_covid)
plt.figure(figsize=(16,12))
plt.imshow(cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
covid_mentions = []
for each in covid_tweets:
   for mention in each._json['entities']['user_mentions']:
        covid_mentions.append(mention['name'])
people_dict=Counter(covid_mentions)

In [ ]:
cloud = WordCloud(max_font_size=80,background_color='white',colormap="viridis").generate_from_frequencies(people_dict)
plt.figure(figsize=(16,12))
plt.imshow(cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# This is taken from data scientist Eugenia Inzaugarat on github https://github.com/ugis22/analysing_twitter/blob/master/Jupyter%20Notebook%20files/Interaction%20Network.ipynb
# Get the interactions between the different users
def get_interactions(row):
    # From every row of the original dataframe
    # First we obtain the 'user_id' and 'screen_name'
    user = row["id"], row["user"]["screen_name"]
    # Be careful if there is no user id
    if user[0] is None:
        return (None, None), []
    
    # The interactions are going to be a set of tuples
    interactions = set()
    
    # Add all interactions 
    # First, we add the interactions corresponding to replies adding the id and screen_name
    interactions.add((row["in_reply_to_user_id"], row["in_reply_to_screen_name"]))
    # After that, we add the interactions with retweets
    interactions.add((row["retweeted_status"]["user"]["id_str"], row["retweeted_status"]["user"]["name"]))
    # And later, the interactions with user mentions
    #interactions.add((row["user_mentions_id"], row["user_mentions_screen_name"]))
    
    # Discard if user id is in interactions
    interactions.discard((row["id"], row["user"]["screen_name"]))
    # Discard all not existing values
    interactions.discard((None, None))
    # Return user and interactions
    return user, interactions

In [ ]:
import networkx as nx
graph = nx.Graph()
covid_tweets = []
for page in tweepy.Cursor(api.search, q='brexit', lang='en', min_retweets="1000").pages(100):
    covid_tweets.append(page)

In [ ]:
for page in covid_tweets:
    for tweet in page:
        try:
            retweet_status = tweet._json["retweeted_status"]
            user, interactions = get_interactions(tweet._json)
            user_id, user_name = user
            #print(user_id)
            tweet_id = tweet._json["id_str"]
            #print(tweet_id)
            for interaction in interactions:
                interaction
                int_id, int_name = interaction
                graph.add_edge(user_id, int_id, tweet_id=tweet_id)    
                graph.node[user_id]["name"] = user_name
                graph.node[int_id]["name"] = int_name
        except:
            pass

In [ ]:
print(f"There are {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges present in the Graph")

In [ ]:
from operator import itemgetter
graph_centrality = nx.degree_centrality(graph)
max_de = max(graph_centrality.items(), key=itemgetter(1))
graph_closeness = nx.closeness_centrality(graph)
max_clo = max(graph_closeness.items(), key=itemgetter(1))
graph_betweenness = nx.betweenness_centrality(graph, normalized=True, endpoints=False)
max_bet = max(graph_betweenness.items(), key=itemgetter(1))

In [ ]:
print(f"the node with id {max_de[0]} has a degree centrality of {max_de[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_clo[0]} has a closeness centrality of {max_clo[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_bet[0]} has a betweenness centrality of {max_bet[1]:.2f} which is the maximum of the Graph")

In [ ]:
node_and_degree = graph.degree()
colors_central_nodes = ['orange', 'red']
central_nodes = ['2334732225', '2334732225']
pos = nx.spring_layout(graph, k=0.05)

In [ ]:
plt.figure(figsize = (20,20))
nx.draw(graph, pos=pos, cmap=plt.cm.PiYG, edge_color="black", linewidths=0.3, node_size=60, alpha=0.6, with_labels=False)
nx.draw_networkx_nodes(graph, pos=pos, nodelist=central_nodes, node_size=300, node_color=colors_central_nodes)
#plt.savefig('graphfinal.png')
plt.show()

In [ ]:
api.get_user('2334732225')

In [ ]:
new_tweets = api.user_timeline(id = '2334732225',count=5)
for tweet in new_tweets:
    print(tweet._json['text'])
    